In [1]:
import json
import pandas as pd
import numpy as np
import requests 
import os
import sys
import pickle
from scipy import stats
from pandas_datareader.famafrench import get_available_datasets
import pandas_datareader.data as web
from yahoo_fin import stock_info as si
from performance_analysis import annualized_return
from performance_analysis import annualized_standard_deviation
from performance_analysis import max_drawdown
from performance_analysis import gain_to_pain_ratio
from performance_analysis import calmar_ratio
from performance_analysis import sharpe_ratio
from performance_analysis import sortino_ratio
import yfinance as yf
import apiclient.discovery
from oauth2client.service_account import ServiceAccountCredentials
import httplib2
from sklearn import mixture as mix
import seaborn as sns 
import gspread as gd

pd.options.mode.chained_assignment = None  # default='warn'


In [21]:
# подключение к гугл таблице

LIST = 'Китай'

index = '^AXJO'

exchange = 'HKSE:0'

exchange_yahoo =  '.HK'

# tickers = si.tickers_sp500()
cheked_year = '2015' 
cheked_year_end = '2020' 


# Файл, полученный в Google Developer Console
CREDENTIALS_FILE = 'Seetzzz-1cb93f64d8d7.json'
# ID Google Sheets документа (можно взять из его URL)
spreadsheet_id = '1lDhu6-tBmoh66a1mY3RU2yPV2_3uIzNSQWNI5UtMcag'
spreadsheet_id2 = '1A3leW6ZfsoVEPXZsv0Loj4eAbyKRchnHrJLdP4RIXDA'
#
# Авторизуемся и получаем service — экземпляр доступа к API
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    CREDENTIALS_FILE,
    ['https://www.googleapis.com/auth/spreadsheets',
     'https://www.googleapis.com/auth/drive'])
httpAuth = credentials.authorize(httplib2.Http())
service = apiclient.discovery.build('sheets', 'v4', http=httpAuth)

# ____________________________Парсим тикеры !!!!С ТАБЛИЦЫ!!!! и работаем с ними _______________________________________

# Чтения файла
values = service.spreadsheets().values().get(
    spreadsheetId=spreadsheet_id,
    range=f'{LIST}!A1:AA1000',
    majorDimension='COLUMNS'
).execute()

tickers = values['values'][7][2:402]

print(tickers)

['HKSE:04338', 'HKSE:00700', 'HKSE:09988', 'HKSE:04335', 'HKSE:01398', 'HKSE:04333', 'HKSE:03968', 'HKSE:00939', 'HKSE:03690', 'HKSE:01288', 'HKSE:02318', 'HKSE:01299', 'HKSE:04332', 'HKSE:00941', 'HKSE:03988', 'HKSE:00857', 'HKSE:04336', 'HKSE:09618', 'HKSE:00005', 'HKSE:01211', 'HKSE:02628', 'HKSE:00388', 'HKSE:01810', 'HKSE:01658', 'HKSE:06288', 'HKSE:02333', 'HKSE:00386', 'HKSE:02269', 'HKSE:09999', 'HKSE:09633', 'HKSE:02359', 'HKSE:02378', 'HKSE:02020', 'HKSE:01088', 'HKSE:09888', 'HKSE:09626', 'HKSE:06030', 'HKSE:03328', 'HKSE:00883', 'HKSE:00016', 'HKSE:00669', 'HKSE:00945', 'HKSE:02202', 'HKSE:02899', 'HKSE:01024', 'HKSE:06690', 'HKSE:02601', 'HKSE:00981', 'HKSE:01772', 'HKSE:01919', 'HKSE:00066', 'HKSE:00011', 'HKSE:01876', 'HKSE:00914', 'HKSE:01339', 'HKSE:02388', 'HKSE:00267', 'HKSE:00175', 'HKSE:02313', 'HKSE:06969', 'HKSE:02382', 'HKSE:02331', 'HKSE:00998', 'HKSE:00003', 'HKSE:06618', 'HKSE:00728', 'HKSE:00001', 'HKSE:00960', 'HKSE:03908', 'HKSE:00027', 'HKSE:06160', 'HKSE

In [17]:
yahoo_ticker_list_full = []

for tic in tickers:
    yahoo_ticker_list_full.append(tic.replace(exchange,'')+exchange_yahoo )

    
price_yahoo_pre_main = yf.download(yahoo_ticker_list_full)
price_yahoo_pre_main = price_yahoo_pre_main['Adj Close'].fillna(method='ffill').fillna(0)

company_yahoo_found = price_yahoo_pre_main.sum()[(price_yahoo_pre_main.sum()!=0)].index.tolist()

tickers = []
# tickers_ttt = 
for y_comp in company_yahoo_found:
    tickers.append(exchange + y_comp.replace(exchange_yahoo, ''))
# price_yahoo_main = yf.download(company_yahoo_found)
# price_yahoo_main = price_yahoo_main['Adj Close'].fillna(method='ffill')

[*********************100%***********************]  400 of 400 completed


# Читаем данные из огурцов

In [19]:
# # ['annuals']

Data_for_Portfolio_TOTAL = pd.DataFrame()



for ticker in tickers:
    with open(f'''C:/Users/Anton/Desktop/Backtesting/BANKA/data_json_{ticker.replace(exchange, '')}.pickle''', 'rb') as f:
        data_json = pickle.load(f)

    with open(f'''C:/Users/Anton/Desktop/Backtesting/BANKA/data_json_keyratios_{ticker.replace(exchange, '')}.pickle''', 'rb') as f:
        data_json_keyratios = pickle.load(f)

#     print(data_json)    
    try:
        date_list = pd.Series(data_json['financials']['annuals']['Fiscal Year'])
        keyratios = pd.DataFrame(data_json_keyratios['Fundamental'], index=[0]) 
        income_df = pd.DataFrame(data_json['financials']['annuals']['income_statement']).set_index(date_list).replace('No Debt', 0).replace('At Loss', 0).replace('-', 0).replace('', 0).replace('N/A', 0).astype(float)
        balance_df = pd.DataFrame(data_json['financials']['annuals']['balance_sheet']).set_index(date_list).replace('No Debt', 0).replace('At Loss', 0).replace('', 0).replace('-', 0).replace('N/A', 0).astype(float)
        cashflow_df = pd.DataFrame(data_json['financials']['annuals']['cashflow_statement']).set_index(date_list).replace('No Debt', 0).replace('At Loss', 0).replace('', 0).replace('-', 0).replace('N/A', 0).astype(float)
        valuation_ratios_df = pd.DataFrame(data_json['financials']['annuals']['valuation_ratios']).set_index(date_list).replace('No Debt', 0).replace('At Loss', 0).replace('', 0).replace('-', 0).replace('N/A', 0).astype(float)
        valuation_and_quality_df = pd.DataFrame(data_json['financials']['annuals']['valuation_and_quality']).set_index(date_list).drop(['Restated Filing Date', 'Filing Date', 'Earnings Release Date'], axis=1).replace('', 0).replace('No Debt', 0).replace('At Loss', 0).replace('-', 0).replace('N/A', 0).astype(float)
        common_size_ratios_df = pd.DataFrame(data_json['financials']['annuals']['common_size_ratios']).set_index(date_list).replace('No Debt', 0).replace('At Loss', 0).replace('', 0).replace('-', 0).replace('N/A', 0).replace('Negative Tangible Equity', 0).astype(float)
        # per_share_data_array_df = pd.DataFrame(data_json['financials']['annuals']['per_share_data_array']).set_index(date_list).replace('-', 0).replace('N/A', 0).astype(float)
        per_share_data_df = pd.DataFrame(data_json['financials']['annuals']['per_share_data_array']).set_index(date_list).replace('', 0).replace('No Debt', 0).replace('-', 0).replace('N/A', 0).astype(float)
        
        check = 1
    except:
        check = 0
        print(ticker)
        
        pass
            
    if check == 1:
    
        try:
            Data_for_Portfolio = pd.DataFrame()
        #     
            Data_for_Portfolio['E/P'] = income_df['Net Income'] / valuation_and_quality_df['Market Cap'] 
            # income_df['Net Income']
            # valuation_and_quality_df.replace('-', 0)


#             Data_for_Portfolio['EBITDA/EV'] = income_df['EBITDA'] / (valuation_and_quality_df['Enterprise Value ($M)']*1000000)
            
            Data_for_Portfolio['EBITDA/EV'] = 1/valuation_ratios_df['EV-to-EBITDA']



            Data_for_Portfolio['ncfcommon'] = cashflow_df['Free Cash Flow']/valuation_and_quality_df['Shares Outstanding (EOP)']

            Data_for_Portfolio['Total Debt'] = (balance_df['Short-Term Debt'] + balance_df['Long-Term Debt'])


            Data_for_Portfolio['Shareholder Yield'] = -((Data_for_Portfolio['Total Debt'] + cashflow_df['Free Cash Flow'] \
                                             + Data_for_Portfolio['ncfcommon']) / valuation_and_quality_df['Market Cap'])


            Data_for_Portfolio['FCF/P'] = cashflow_df['Free Cash Flow'] / valuation_and_quality_df['Market Cap']
            Data_for_Portfolio
            # Data_for_Portfolio['Shareholder Yield'] 
            Data_for_Portfolio['Book Value per Share'] = per_share_data_df['Book Value per Share']
            Data_for_Portfolio['Dividends per Share'] =per_share_data_df['Dividends per Share'] 
            Data_for_Portfolio['Dividend Payout Ratio'] =common_size_ratios_df['Dividend Payout Ratio']

            Data_for_Portfolio['FCF/Assets'] = cashflow_df['Free Cash Flow'] / balance_df['Total Current Assets']

            #Can you generate returns on investment?   



            Data_for_Portfolio['ROA'] = common_size_ratios_df['ROA %']  
            Data_for_Portfolio['ROIC'] = common_size_ratios_df['ROIC %']
            #Do you have a defendable business model?

            Data_for_Portfolio['GROSS MARGIN'] = common_size_ratios_df['Gross Margin %']

            Data_for_Portfolio['CURRENT RATIO'] = balance_df['Total Current Assets'] / balance_df['Total Current Liabilities']




            Data_for_Portfolio['INTEREST/EBITDA'] = income_df['Interest Expense'] / income_df['EBITDA']





            Data_for_Portfolio['Company'] = ticker
            Data_for_Portfolio['Date'] = list(date_list)

            Data_for_Portfolio = Data_for_Portfolio.replace([np.inf, -np.inf], 0)
            Data_for_Portfolio = Data_for_Portfolio.set_index('Company')
            Data_for_Portfolio = Data_for_Portfolio[::-1]

            sumz_frame= [Data_for_Portfolio, Data_for_Portfolio_TOTAL]
            Data_for_Portfolio_TOTAL = pd.concat(sumz_frame)

        except:
            pass

#         print(list(date_list))
# list(set(Data_for_Portfolio_TOTAL.index.tolist()))

In [20]:
len(list(set(Data_for_Portfolio_TOTAL.index.tolist())))

342

In [6]:
# list(set(Data_for_Portfolio_TOTAL.index.tolist()))

In [7]:
# # # ['annuals']

# tickers = si.tickers_sp500()

# Data_for_Portfolio_TOTAL = pd.DataFrame()



# for ticker in tickers:
#     with open(f'BANKA/data_json_{ticker}.pickle', 'rb') as f:
#         data_json = pickle.load(f)

#     with open(f'BANKA/data_json_keyratios_{ticker}.pickle', 'rb') as f:
#         data_json_keyratios = pickle.load(f)

# #     print(data_json)    
#     try:
#         date_list = pd.Series(data_json['financials']['annuals']['Fiscal Year'])
#         keyratios = pd.DataFrame(data_json_keyratios['Fundamental'], index=[0]) 
#         income_df = pd.DataFrame(data_json['financials']['annuals']['income_statement']).set_index(date_list).replace('No Debt', 0).replace('At Loss', 0).replace('-', 0).replace('', 0).replace('N/A', 0).astype(float)
#         balance_df = pd.DataFrame(data_json['financials']['annuals']['balance_sheet']).set_index(date_list).replace('No Debt', 0).replace('At Loss', 0).replace('', 0).replace('-', 0).replace('N/A', 0).astype(float)
#         cashflow_df = pd.DataFrame(data_json['financials']['annuals']['cashflow_statement']).set_index(date_list).replace('No Debt', 0).replace('At Loss', 0).replace('', 0).replace('-', 0).replace('N/A', 0).astype(float)
#         valuation_ratios_df = pd.DataFrame(data_json['financials']['annuals']['valuation_ratios']).set_index(date_list).replace('No Debt', 0).replace('At Loss', 0).replace('', 0).replace('-', 0).replace('N/A', 0).astype(float)
#         valuation_and_quality_df = pd.DataFrame(data_json['financials']['annuals']['valuation_and_quality']).set_index(date_list).drop(['Restated Filing Date', 'Filing Date', 'Earnings Release Date'], axis=1).replace('', 0).replace('No Debt', 0).replace('At Loss', 0).replace('-', 0).replace('N/A', 0).astype(float)
#         common_size_ratios_df = pd.DataFrame(data_json['financials']['annuals']['common_size_ratios']).set_index(date_list).replace('No Debt', 0).replace('At Loss', 0).replace('', 0).replace('-', 0).replace('N/A', 0).replace('Negative Tangible Equity', 0).astype(float)
#         # per_share_data_array_df = pd.DataFrame(data_json['financials']['annuals']['per_share_data_array']).set_index(date_list).replace('-', 0).replace('N/A', 0).astype(float)
#         per_share_data_df = pd.DataFrame(data_json['financials']['annuals']['per_share_data_array']).set_index(date_list).replace('', 0).replace('No Debt', 0).replace('-', 0).replace('N/A', 0).astype(float)
        
#         check = 1
#     except:
#         pass
            
#     if check == 1:
    
#         Data_for_Portfolio = pd.DataFrame()
#     #     
#         Data_for_Portfolio['E/P'] = income_df['Net Income'] / valuation_and_quality_df['Market Cap'] 
#         # income_df['Net Income']
#         # valuation_and_quality_df.replace('-', 0)
#         Data_for_Portfolio
#         try:
#             Data_for_Portfolio['EBITDA/EV'] = income_df['EBITDA'] / (valuation_and_quality_df['Enterprise Value ($M)']*1000000)
#         except:
#             Data_for_Portfolio['EBITDA/EV'] = Data_for_Portfolio['E/P'] * 0

#         Data_for_Portfolio['ncfcommon'] = cashflow_df['Free Cash Flow']/valuation_and_quality_df['Shares Outstanding (EOP)']
#         try:
#             Data_for_Portfolio['Total Debt'] = (balance_df['Short-Term Debt'] + balance_df['Long-Term Debt'])
#         except:
#             Data_for_Portfolio['Total Debt'] = Data_for_Portfolio['ncfcommon'] * 0
# #             print(Data_for_Portfolio)

#         Data_for_Portfolio['Shareholder Yield'] = -((Data_for_Portfolio['Total Debt'] + cashflow_df['Free Cash Flow'] \
#                                          + Data_for_Portfolio['ncfcommon']) / valuation_and_quality_df['Market Cap'])


#         Data_for_Portfolio['FCF/P'] = cashflow_df['Free Cash Flow'] / valuation_and_quality_df['Market Cap']
#         Data_for_Portfolio
#         # Data_for_Portfolio['Shareholder Yield'] 
#         Data_for_Portfolio['Book Value per Share'] = per_share_data_df['Book Value per Share']
#         Data_for_Portfolio['Dividends per Share'] =per_share_data_df['Dividends per Share'] 
#         Data_for_Portfolio['Dividend Payout Ratio'] =common_size_ratios_df['Dividend Payout Ratio']

#         try:
#             Data_for_Portfolio['FCF/Assets'] = cashflow_df['Free Cash Flow'] / balance_df['Total Current Assets']
#         except:
#             Data_for_Portfolio['FCF/Assets'] = cashflow_df['Free Cash Flow'] / balance_df['Total Assets']
#         #Can you generate returns on investment?   



#         Data_for_Portfolio['ROA'] = common_size_ratios_df['ROA %']  
#         Data_for_Portfolio['ROIC'] = common_size_ratios_df['ROIC %']
#         #Do you have a defendable business model?
#         try:
#             Data_for_Portfolio['GROSS MARGIN'] = common_size_ratios_df['Gross Margin %']
#         except:
#             Data_for_Portfolio['GROSS MARGIN'] = Data_for_Portfolio['ROA'] * 0
#         #Current Financial Strength
#         try:
#             Data_for_Portfolio['CURRENT RATIO'] = balance_df['Total Current Assets'] / balance_df['Total Current Liabilities']
#         except:
#             Data_for_Portfolio['CURRENT RATIO'] = balance_df['Total Assets'] / balance_df['Total Liabilities']


#         try:
#             Data_for_Portfolio['INTEREST/EBITDA'] = income_df['Interest Expense'] / income_df['EBITDA']
#         except:
#             try:
#                 Data_for_Portfolio['INTEREST/EBITDA'] = income_df['Interest Expense (Positive)'] / income_df['EBITDA']
#             except:
#                 Data_for_Portfolio['INTEREST/EBITDA'] = income_df['Interest Expense'] / 0




#         Data_for_Portfolio['Company'] = ticker
#         Data_for_Portfolio['Date'] = list(date_list)

#         Data_for_Portfolio = Data_for_Portfolio.replace([np.inf, -np.inf], 0)
#         Data_for_Portfolio = Data_for_Portfolio.set_index('Company')
#         Data_for_Portfolio = Data_for_Portfolio[::-1]

#         sumz_frame= [Data_for_Portfolio, Data_for_Portfolio_TOTAL]
#         Data_for_Portfolio_TOTAL = pd.concat(sumz_frame)



# #         print(list(date_list))


In [22]:
# tickers = ['AAPL', 'INTC']

yahoo_ticker_list_full = []

for tic in tickers:
    yahoo_ticker_list_full.append(tic.replace(exchange,'')+exchange_yahoo )

price_yahoo_main = yf.download(yahoo_ticker_list_full)
price_yahoo_main = price_yahoo_main['Adj Close'].fillna(method='backfill')




years_len = int(cheked_year_end) - int(cheked_year)

portfolio_profit_final = []
index_profit_final = []
max_dd_list = []


Percentile_split = .2

Winsorize_Threshold = .025

# for i in range(years_len):
for i in range(years_len):
    print('i'*50)
    print(i)

    df_res = pd.DataFrame()
    
    for ticker in tickers:
        try:

            Data_for_Portfolio_tick = Data_for_Portfolio_TOTAL.loc[ticker].fillna(0).iloc[int(cheked_year_end) - (int(cheked_year)-1+i)]
#             print(Data_for_Portfolio_tick)
            sum_frame = [pd.DataFrame([Data_for_Portfolio_tick]), df_res]
            df_res = pd.concat(sum_frame )  
            
        except:
            pass
        
    
    Data_for_Portfolio_master_filter = df_res
    
    yahoo_ticker_list = []

    for tic in Data_for_Portfolio_master_filter.index.tolist():
        yahoo_ticker_list.append(tic.replace(exchange,'')+exchange_yahoo )
         

   #Winsorize the metric data and compress outliers if desired
    Data_for_Portfolio_master_filter['E/P Winsorized'] = \
        stats.mstats.winsorize(Data_for_Portfolio_master_filter['E/P'], \
                               limits=Winsorize_Threshold)
    Data_for_Portfolio_master_filter['EBITDA/EV Winsorized'] = \
        stats.mstats.winsorize(Data_for_Portfolio_master_filter['EBITDA/EV'], \
                               limits=Winsorize_Threshold)
    Data_for_Portfolio_master_filter['FCF/P Winsorized'] = \
        stats.mstats.winsorize(Data_for_Portfolio_master_filter['FCF/P'], \
                               limits=Winsorize_Threshold)

    #create Z score to normalize the metrics
    Data_for_Portfolio_master_filter['E/P Z score'] = \
        stats.zscore(Data_for_Portfolio_master_filter['E/P Winsorized'])
    Data_for_Portfolio_master_filter['EBITDA/EV Z score'] = \
        stats.zscore(Data_for_Portfolio_master_filter['EBITDA/EV Winsorized'])
    Data_for_Portfolio_master_filter['FCF/P Z score'] = \
        stats.zscore(Data_for_Portfolio_master_filter['FCF/P Winsorized'])

    Data_for_Portfolio_master_filter['Valuation Score'] = \
            Data_for_Portfolio_master_filter['E/P Z score'] \
            + Data_for_Portfolio_master_filter['EBITDA/EV Z score']\
            + Data_for_Portfolio_master_filter['FCF/P Z score']

    Data_for_Portfolio_master_filter['FCF/P Winsorized']

    ###### QUALITY FACTOR ######  
    Data_for_Portfolio_master_filter = Data_for_Portfolio_master_filter.replace([np.inf, -np.inf], 0)


    Data_for_Portfolio_master_filter['FCF/Assets Winsorized'] = \
        stats.mstats.winsorize(Data_for_Portfolio_master_filter['FCF/Assets'], \
                               limits=Winsorize_Threshold)
    Data_for_Portfolio_master_filter['ROA Winsorized'] = \
        stats.mstats.winsorize(Data_for_Portfolio_master_filter['ROA'], \
                               limits=Winsorize_Threshold)
    Data_for_Portfolio_master_filter['ROIC Winsorized'] = \
        stats.mstats.winsorize(Data_for_Portfolio_master_filter['ROIC'], \
                                limits=Winsorize_Threshold)
    Data_for_Portfolio_master_filter['Gross Margin Winsorized'] = \
        stats.mstats.winsorize(Data_for_Portfolio_master_filter['GROSS MARGIN'], \
                               limits=Winsorize_Threshold)
    Data_for_Portfolio_master_filter['Current Ratio Winsorized'] = \
        stats.mstats.winsorize(Data_for_Portfolio_master_filter['CURRENT RATIO'], \
                               limits=Winsorize_Threshold)
    Data_for_Portfolio_master_filter['Interest/EBITDA Winsorized'] = \
        stats.mstats.winsorize(Data_for_Portfolio_master_filter['INTEREST/EBITDA'], \
                               limits=Winsorize_Threshold)

    #create Z score
    Data_for_Portfolio_master_filter = Data_for_Portfolio_master_filter.replace([np.inf, -np.inf], 0)


    Data_for_Portfolio_master_filter['FCF/Assets Z score'] = \
        stats.zscore(Data_for_Portfolio_master_filter['FCF/Assets Winsorized'])
    Data_for_Portfolio_master_filter['ROA Z score'] = \
        stats.zscore(Data_for_Portfolio_master_filter['ROA Winsorized'])
    Data_for_Portfolio_master_filter['ROIC Z score'] = \
        stats.zscore(Data_for_Portfolio_master_filter['ROIC Winsorized'])
    Data_for_Portfolio_master_filter['Gross Margin Z score'] = \
        stats.zscore(Data_for_Portfolio_master_filter['Gross Margin Winsorized'])
    Data_for_Portfolio_master_filter['Current Ratio Z score'] = \
        stats.zscore(Data_for_Portfolio_master_filter['Current Ratio Winsorized'])
    Data_for_Portfolio_master_filter['Interest/EBITDA Z score'] = \
        stats.zscore(Data_for_Portfolio_master_filter['Interest/EBITDA Winsorized'])

    Data_for_Portfolio_master_filter['Quality Score'] = \
        Data_for_Portfolio_master_filter['FCF/Assets Z score'] \
            + Data_for_Portfolio_master_filter['ROA Z score'] \
            + Data_for_Portfolio_master_filter['ROIC Z score']\
            + Data_for_Portfolio_master_filter['Gross Margin Z score']\
            + Data_for_Portfolio_master_filter['Current Ratio Z score']\
            - Data_for_Portfolio_master_filter['Interest/EBITDA Z score']


    
   
    
    ###### SHAREHOLDER YIELD FACTOR #####

    Data_for_Portfolio_master_filter = Data_for_Portfolio_master_filter.replace([np.inf, -np.inf], 0)

    Data_for_Portfolio_master_filter['Shareholder Yield Winsorized'] = \
        stats.mstats.winsorize(Data_for_Portfolio_master_filter['Shareholder Yield'], \
                               limits=Winsorize_Threshold)
    Data_for_Portfolio_master_filter['Shareholder Yield Z score'] = \
        stats.zscore(Data_for_Portfolio_master_filter['Shareholder Yield Winsorized'])
    Data_for_Portfolio_master_filter['Shareholder Yield Score'] = \
        Data_for_Portfolio_master_filter['Shareholder Yield Z score'] 
        
        
        ###### LOW VOLATILITY FACTOR ######

    #must have fundamental data from previous factors for price based factors
    #as some equities have price data and no fundamental data which should not
    #be included

    # treasury = 'RGBI.ME'
    start = cheked_year
    # end = current_date
    end =  cheked_year_end

#     price_yahoo = yf.download(Data_for_Portfolio_master_filter.index.tolist())
    price_yahoo = price_yahoo_main[yahoo_ticker_list]

    Sector_stock_returns =  price_yahoo.pct_change()      

    #create rolling vol metric for previous 2 years
    Sector_stock_rolling_vol = Sector_stock_returns.rolling(252*2).std()

    #Choose second to last trading day to look at previous vol   
    #Sometimes the dates are off when trying to line up end of quarter and business
    #days so to eliminate errors in the for loop I go to day of quarter, shift forward
    #a business day and then go back two business days

    

    Filter_Vol_Signal = Sector_stock_rolling_vol[str(int(start)+i)].dropna()
    
    

    # Filter_Vol_Signal_Sort = Filter_Vol_Signal.sort_values().dropna() # для цикла паска тикеров

    #create z score and rank for the Volatility Factor
    # frame = { 'Vol': Filter_Vol_Signal} 

    # Filter_Vol_Signal_df = pd.DataFrame(frame)
    
    
#     print(pd.DataFrame(stats.zscore(Filter_Vol_Signal)).mean())
#     print(pd.DataFrame(stats.zscore(Filter_Vol_Signal)))

#     Filter_Vol_Signal = Filter_Vol_Signal.fillna(0)

    # Filter_Vol_Signal_df['Vol Z Score'] = stats.zscore(Filter_Vol_Signal) 
    # Filter_Vol_Signal_df = Filter_Vol_Signal_df.reset_index()
    # print(Filter_Vol_Signal_df)

    
    Data_for_Portfolio_master_filter['Vol Z Score'] = pd.DataFrame(stats.zscore(Filter_Vol_Signal)).fillna(0).mean().tolist()
#     print(stats.zscore(Filter_Vol_Signal))
    # Data_for_Portfolio_master_filter = Data_for_Portfolio_master_filter.merge(Filter_Vol_Signal_df, how = 'inner', on = ['ticker']) 

    ###### TREND FACTOR #####
    
    total_trend_score = []
    
    for tic in yahoo_ticker_list:
        try:
            df = yf.download(tic, str(int(start)+i-2)+'-1-1', str(int(start)+i+1)+'-1-1')
        #     print(df)
            df = df[['Open', 'High', 'Low', 'Adj Close']]
            df['open'] = df['Open'].shift(1)
            df['high'] = df['High'].shift(1)
            df['low'] = df['Low'].shift(1)
            df['close'] = df['Adj Close'].shift(1)

            df = df[['open', 'high', 'low', 'close']]
            df = df.dropna()

            unsup = mix.GaussianMixture(n_components=4,
                                        covariance_type="spherical",
                                        n_init=100,
                                        random_state=42)
            unsup.fit(np.reshape(df, (-1, df.shape[1])))
            regime = unsup.predict(np.reshape(df, (-1, df.shape[1])))
            df['Return'] = np.log(df['close'] / df['close'].shift(1))
            Regimes = pd.DataFrame(regime, columns=['Regime'], index=df.index) \
                .join(df, how='inner') \
                .assign(market_cu_return=df.Return.cumsum()) \
                .reset_index(drop=False) \
                .rename(columns={'index': 'Date'})

            order = [0, 1, 2, 3]
        #     fig = sns.FacetGrid(data=Regimes, hue='Regime', hue_order=order, aspect=2, height=4)
        #     fig.map(plt.scatter, 'Date', 'market_cu_return', s=4).add_legend()
        #     plt.show()

            mean_for_regime = []
            cur_price = df['close'][-1]

            total_position = 0

            for j in order:
                mean_for_regime.append(unsup.means_[j][0])
    #             print('Mean for regime %i: '%i,unsup.means_[i][0])
        #         print('Co-Variance for regime %i: '%i,(unsup.covariances_[i]))

            mean_for_regime = np.sort(mean_for_regime)   
            for val in  mean_for_regime:
                if cur_price > val:
                    total_position += 0.25
                else:
                    pass
                
            total_trend_score.append(total_position)
    #         print(mean_for_regime)
    #         print('cur_price')
    #         print(cur_price)
    #         print('total_position')
    #         print(total_position)
        except:
            total_trend_score.append(0)

        
    


    Data_for_Portfolio_master_filter['Trend Score'] = total_trend_score
    
    #This is a very simply way to see how much a stock is in a trend up or down
    #You could easily make this more complex/robust but it would cost you in 
    #execution time
#     df_sma_50 = price_yahoo.rolling(50).mean()
#     df_sma_100 = price_yahoo.rolling(100).mean()
#     df_sma_150 = price_yahoo.rolling(150).mean()
#     df_sma_200 = price_yahoo.rolling(200).mean()


#     Filter_Trend_Signal_50 = df_sma_50[str(int(cheked_year)+i)]
#     Filter_Trend_Signal_100 = df_sma_100[str(int(cheked_year)+i)]
#     Filter_Trend_Signal_150 = df_sma_150[str(int(cheked_year)+i)]
#     Filter_Trend_Signal_200 = df_sma_200[str(int(cheked_year)+i)]

#     Price_Signal = price_yahoo[str(int(cheked_year)+i)]

#     SMA_all = pd.DataFrame()
#     SMA_50 = pd.DataFrame(np.where(Price_Signal > Filter_Trend_Signal_50,1,0)).mean()
#     SMA_100 = pd.DataFrame(np.where(Price_Signal > Filter_Trend_Signal_100,1,0)).mean()
#     SMA_150 = pd.DataFrame(np.where(Price_Signal > Filter_Trend_Signal_150,1,0)).mean()
#     SMA_200 = pd.DataFrame(np.where(Price_Signal > Filter_Trend_Signal_200,1,0)).mean()

#     SMA_all['SMA_50'] = SMA_50
#     SMA_all['SMA_100'] = SMA_100
#     SMA_all['SMA_150'] = SMA_150
#     SMA_all['SMA_200'] = SMA_200
#     SMA_all['Trend Score'] = np.mean(SMA_all, axis=1)

#     # print(SMA_all)
#     Data_for_Portfolio_master_filter['Trend Score'] = np.mean(SMA_all, axis=1).tolist()


    
    
    

    ###### MOMENTUM FACTOR #####

#     print('tut')
    
    # tickers_momentum = list(Sector_stock_prices_vol_df_1_wide.columns)
    #from the academic literature of 12 months - 1 month momentum 
#     df_mom_11_months = price_yahoo[str(int(cheked_year)+i)].pct_change(22*11)
#     Data_for_Portfolio_master_filter['Momentum Score'] = pd.DataFrame(stats.zscore(df_mom_11_months.iloc[242:])).fillna(0).mean().tolist()
#     # Data_for_Portfolio_master_filter = Data_for_Portfolio_master_filter.merge(Filter_MOM_df[['ticker','Momentum Score']], how = 'inner', on = ['ticker'])
#     Data_for_Portfolio_master_filter



    prices = price_yahoo_main[yahoo_ticker_list].asfreq('BM')
    prices_yearly_returns = prices.pct_change(12)
    prices_yearly_signal = np.where(prices_yearly_returns[str(int(cheked_year)+i)].iloc[-1] > 0, 1, 0)
    Data_for_Portfolio_master_filter['Momentum Score'] = prices_yearly_signal

    
       
    ### Create Composite Score from factors ###

    #Because we made all the factors with a z score each factor should have equal
    #weight in the composite. You could consider changing the weights based on 
    #historical statistical significance or whatever else seems reasonable

    #This particular scoring system only invests in companies with 
    #positive trend/momentum after ranking by the other factors

    Data_for_Portfolio_master_filter['Total Score'] = Data_for_Portfolio_master_filter['Valuation Score'] +  \
        Data_for_Portfolio_master_filter['Quality Score'] + Data_for_Portfolio_master_filter['Shareholder Yield Score'] +\
        Data_for_Portfolio_master_filter['Momentum Score'] + Data_for_Portfolio_master_filter['Trend Score']
    
    
    
    Data_for_Portfolio_master_filter = Data_for_Portfolio_master_filter.sort_values('Total Score', ascending=False)
    
    top_rated_company = Data_for_Portfolio_master_filter[:int(len(Data_for_Portfolio_master_filter) \
                                                              * Percentile_split)].index.tolist()
    top_rated_company

    low_rated_company = Data_for_Portfolio_master_filter[-int(len(Data_for_Portfolio_master_filter) \
                                                              * Percentile_split):].index.tolist()
    low_rated_company
    
   
    
    
    
    start_hayoo = str(int(start)+i+1)+'-1-1'
    end_hayoo = str(int(start)+i+2)+'-1-1'
    
    cum_str_returns_bh = (price_yahoo[yahoo_ticker_list][str(int(start)+i+1)].fillna(method='backfill').pct_change() + 1).cumprod().fillna(0)
    running_max_BH = np.maximum.accumulate(cum_str_returns_bh[1:].fillna(method='backfill') )
    drawdown_BH = (cum_str_returns_bh[1:])/running_max_BH - 1
    max_dd = drawdown_BH.min()*100


    try:
        Data_for_Portfolio_master_filter['Max DD'] = max_dd.values 
    except:
        Data_for_Portfolio_master_filter['Max DD'] = [max_dd]
    
    max_dd_list.append(max_dd.min())
    
#     # == Доходность
    
    portfolio_profit = [] 
    profit_list_index = 0

    top_rated_company_yahoo = []
    low_rated_company_yahoo = []
    
    
    for tic in top_rated_company:
        top_rated_company_yahoo.append(tic.replace(exchange,'')+exchange_yahoo )
        
    for tic in low_rated_company:
        low_rated_company_yahoo.append(tic.replace(exchange,'')+exchange_yahoo )
        
  
    
#     profit_yah = yf.download(top_rated_company, start_hayoo, end_hayoo)['Adj Close'].fillna(method='backfill') 
    profit_yah = price_yahoo_main[top_rated_company_yahoo][str(int(start)+i+1)].fillna(method='backfill') 
    profit = (profit_yah.iloc[-1]-profit_yah.iloc[0])/profit_yah.iloc[0]
    profit = profit.dropna()
#     profit_yah['profit'] = profit
    portfolio_profit = profit.values.tolist()
#     print(profit_yah)
            
    
#     for company in top_rated_company:
# #         print('tut1')
#         try:
#             profit_yah = yf.download(company, start_hayoo, end_hayoo)['Adj Close'].fillna(method='backfill') 
#             profit = (profit_yah[-1]-profit_yah[0])/profit_yah[0]
#             (1 + profit).cumprod()[-1]
#             portfolio_profit.append(profit)
#             print(profit)
#         except:
#             pass
#     portfolio_profit


#     profit_list_index = 0

    profit_yah_index = yf.download(index)['Adj Close'].fillna(method='backfill')[str(int(start)+i)] 
    profit_index = (profit_yah_index[-1]-profit_yah_index[0])/profit_yah_index[0]

    

    print('Год начальный')
    print(start_hayoo)
    
#     profit_list_index = profit_index

    portfolio_profit_final.append(np.mean(portfolio_profit)*100)
    index_profit_final.append(profit_index*100)

#     returnezzz = pd.DataFrame()
#     returnezzz['Portfolio'] = [np.mean(portfolio_profit)*100]
#     returnezzz['Index'] = [profit_list_index*100]

    print(portfolio_profit_final)

    print('top_rated_company')
    print(top_rated_company)
    print('low_rated_company')
    print(low_rated_company)
       

    print('Max DD')
    print(max_dd_list)
    print(np.min(max_dd_list))




[*********************100%***********************]  400 of 400 completed
iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii
0


<ipython-input-22-7b8f81555159>:169: FutureWarning: Indexing a DataFrame with a datetimelike index using a single string to slice the rows, like `frame[string]`, is deprecated and will be removed in a future version. Use `frame.loc[string]` instead.
  Filter_Vol_Signal = Sector_stock_rolling_vol[str(int(start)+i)].dropna()


[*********************100%***********************]  1 of 1 completed

1 Failed download:
- 3668.HK: Data doesn't exist for startDate = 1356987600, endDate = 1451595600
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- 3990.HK: Data doesn't exist for startDate = 1356987600, endDate = 1451595600
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%*****************

<ipython-input-22-7b8f81555159>:313: FutureWarning: Indexing a DataFrame with a datetimelike index using a single string to slice the rows, like `frame[string]`, is deprecated and will be removed in a future version. Use `frame.loc[string]` instead.
  prices_yearly_signal = np.where(prices_yearly_returns[str(int(cheked_year)+i)].iloc[-1] > 0, 1, 0)
<ipython-input-22-7b8f81555159>:350: FutureWarning: Indexing a DataFrame with a datetimelike index using a single string to slice the rows, like `frame[string]`, is deprecated and will be removed in a future version. Use `frame.loc[string]` instead.
  cum_str_returns_bh = (price_yahoo[yahoo_ticker_list][str(int(start)+i+1)].fillna(method='backfill').pct_change() + 1).cumprod().fillna(0)
<ipython-input-22-7b8f81555159>:381: FutureWarning: Indexing a DataFrame with a datetimelike index using a single string to slice the rows, like `frame[string]`, is deprecated and will be removed in a future version. Use `frame.loc[string]` instead.
  profit_

[*********************100%***********************]  1 of 1 completed
Год начальный
2016-1-1
[11.620573426136147]
top_rated_company
['HKSE:03918', 'HKSE:00336', 'HKSE:01999', 'HKSE:06826', 'HKSE:00696', 'HKSE:04332', 'HKSE:04333', 'HKSE:00460', 'HKSE:04335', 'HKSE:01686', 'HKSE:02020', 'HKSE:06865', 'HKSE:00823', 'HKSE:01051', 'HKSE:03799', 'HKSE:01548', 'HKSE:00973', 'HKSE:01368', 'HKSE:02018', 'HKSE:01177', 'HKSE:02386', 'HKSE:01530', 'HKSE:00014', 'HKSE:04338', 'HKSE:00548', 'HKSE:02128', 'HKSE:00001', 'HKSE:02313', 'HKSE:00247', 'HKSE:00004', 'HKSE:01071', 'HKSE:02518', 'HKSE:01528', 'HKSE:00270', 'HKSE:01456', 'HKSE:03898', 'HKSE:00053', 'HKSE:00836', 'HKSE:00303', 'HKSE:09999', 'HKSE:00006', 'HKSE:01113', 'HKSE:00177', 'HKSE:00268', 'HKSE:00688', 'HKSE:00941', 'HKSE:09888', 'HKSE:00552', 'HKSE:09988', 'HKSE:01972', 'HKSE:03888', 'HKSE:01093', 'HKSE:01038', 'HKSE:01347', 'HKSE:00083', 'HKSE:02380', 'HKSE:01882', 'HKSE:00338', 'HKSE:02638', 'HKSE:01109']
low_rated_company
['HKSE:033

...

[*********************100%***********************]  1 of 1 completed
Год начальный
2019-1-1
[11.620573426136147, 56.63261861799894, 5.250894871055298, 33.29917335403222]
top_rated_company
['HKSE:00914', 'HKSE:03668', 'HKSE:00867', 'HKSE:03918', 'HKSE:00189', 'HKSE:00336', 'HKSE:04332', 'HKSE:01313', 'HKSE:01177', 'HKSE:00883', 'HKSE:00460', 'HKSE:03799', 'HKSE:00823', 'HKSE:00323', 'HKSE:04336', 'HKSE:00014', 'HKSE:06826', 'HKSE:00548', 'HKSE:01579', 'HKSE:01972', 'HKSE:04335', 'HKSE:02020', 'HKSE:04338', 'HKSE:02518', 'HKSE:03319', 'HKSE:01928', 'HKSE:01113', 'HKSE:01088', 'HKSE:00136', 'HKSE:01252', 'HKSE:01066', 'HKSE:06098', 'HKSE:02338', 'HKSE:02128', 'HKSE:03808', 'HKSE:00467', 'HKSE:01171', 'HKSE:00083', 'HKSE:01128', 'HKSE:00270', 'HKSE:01888', 'HKSE:01347', 'HKSE:01997', 'HKSE:01368', 'HKSE:04333', 'HKSE:09992', 'HKSE:00486', 'HKSE:00347', 'HKSE:06288', 'HKSE:03692', 'HKSE:00839', 'HKSE:01958', 'HKSE:02313', 'HKSE:00267', 'HKSE:00586', 'HKSE:03380', 'HKSE:01530', 'HKSE:02778',

<ipython-input-22-7b8f81555159>:169: FutureWarning: Indexing a DataFrame with a datetimelike index using a single string to slice the rows, like `frame[string]`, is deprecated and will be removed in a future version. Use `frame.loc[string]` instead.
  Filter_Vol_Signal = Sector_stock_rolling_vol[str(int(start)+i)].dropna()


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- 2160.HK: Data doesn't exist for startDate = 1483218000, endDate = 1577826000
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- 2599.HK: Data doesn't exist for startDate = 1483218000, endDate = 1577826000
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%*****************

<ipython-input-22-7b8f81555159>:313: FutureWarning: Indexing a DataFrame with a datetimelike index using a single string to slice the rows, like `frame[string]`, is deprecated and will be removed in a future version. Use `frame.loc[string]` instead.
  prices_yearly_signal = np.where(prices_yearly_returns[str(int(cheked_year)+i)].iloc[-1] > 0, 1, 0)
<ipython-input-22-7b8f81555159>:350: FutureWarning: Indexing a DataFrame with a datetimelike index using a single string to slice the rows, like `frame[string]`, is deprecated and will be removed in a future version. Use `frame.loc[string]` instead.
  cum_str_returns_bh = (price_yahoo[yahoo_ticker_list][str(int(start)+i+1)].fillna(method='backfill').pct_change() + 1).cumprod().fillna(0)
<ipython-input-22-7b8f81555159>:381: FutureWarning: Indexing a DataFrame with a datetimelike index using a single string to slice the rows, like `frame[string]`, is deprecated and will be removed in a future version. Use `frame.loc[string]` instead.
  profit_

[*********************100%***********************]  1 of 1 completed
Год начальный
2020-1-1
[11.620573426136147, 56.63261861799894, 5.250894871055298, 33.29917335403222, 30.269907121663415]
top_rated_company
['HKSE:02400', 'HKSE:00914', 'HKSE:00336', 'HKSE:03668', 'HKSE:00867', 'HKSE:03799', 'HKSE:06186', 'HKSE:01313', 'HKSE:03918', 'HKSE:00883', 'HKSE:01579', 'HKSE:03692', 'HKSE:04332', 'HKSE:00014', 'HKSE:00151', 'HKSE:01928', 'HKSE:04333', 'HKSE:02518', 'HKSE:04335', 'HKSE:00189', 'HKSE:00973', 'HKSE:06826', 'HKSE:00303', 'HKSE:01113', 'HKSE:01088', 'HKSE:01378', 'HKSE:01177', 'HKSE:01958', 'HKSE:04338', 'HKSE:00564', 'HKSE:03319', 'HKSE:00316', 'HKSE:01171', 'HKSE:02020', 'HKSE:01530', 'HKSE:00053', 'HKSE:03998', 'HKSE:00027', 'HKSE:00486', 'HKSE:06098', 'HKSE:01252', 'HKSE:01368', 'HKSE:06288', 'HKSE:01066', 'HKSE:03323', 'HKSE:03808', 'HKSE:09992', 'HKSE:00152', 'HKSE:03868', 'HKSE:01999', 'HKSE:00016', 'HKSE:01093', 'HKSE:02331', 'HKSE:01157', 'HKSE:00270', 'HKSE:02313', 'HKSE:0

In [23]:
returnez_cum_port = pd.DataFrame(portfolio_profit_final).dropna()  
returnez_cum_index = pd.DataFrame(index_profit_final).dropna()  

returnez = pd.DataFrame()

returnez['Страна'] = [LIST]
returnez['Начало периода'] = [cheked_year]
returnez['Дходность с ребалансировкой портфеля'] = ((1 + (returnez_cum_port/100)).cumprod().iloc[-1]-1)*100
returnez['Дходность Индекса'] = ((1 + (returnez_cum_index/100)).cumprod().iloc[-1]-1)*100
returnez['Max DD'] = [np.min(max_dd_list)]

# gc = gd.service_account(filename='Seetzzz-1cb93f64d8d7.json')
# worksheet = gc.open("Тесты бэктестинга").worksheet('Мульти-фактор2')

# worksheet.update('A20', [returnez.columns.tolist()] + returnez.values.tolist())

returnez

,Страна,Начало периода,Дходность с ребалансировкой портфеля,Дходность Индекса,Max DD
0,Китай,2015,219.538958,24.126498,-88.069458


In [24]:
Data_for_Portfolio_master_filter

,E/P,EBITDA/EV,ncfcommon,Total Debt,Shareholder Yield,FCF/P,Book Value per Share,Dividends per Share,Dividend Payout Ratio,FCF/Assets,...,Interest/EBITDA Z score,Quality Score,Shareholder Yield Winsorized,Shareholder Yield Z score,Shareholder Yield Score,Vol Z Score,Trend Score,Momentum Score,Total Score,Max DD
HKSE:02400,0.054593,0.131926,1.930559,0.000,-0.113836,0.113562,5.013,0.000,0.000,0.303626,...,0.910372,7.150135,-0.113836,0.579225,0.579225,-1.949660e-16,1.00,1,10.346748,-30.480504
HKSE:00914,0.124187,0.213675,6.690843,11473.622,-0.155937,0.117797,28.843,1.882,0.267,0.328125,...,0.858198,4.741801,-0.155937,0.541854,0.541854,-1.807046e-15,1.00,1,9.995255,0.000000
HKSE:00336,0.139213,0.346021,0.446658,3336.891,-0.531644,0.156174,4.235,0.175,0.440,0.142664,...,0.072607,5.043700,-0.531644,0.208364,0.208364,-1.848566e-15,0.00,0,9.610468,-31.797557
HKSE:03668,0.193569,0.251256,5.141184,18296.500,-1.256709,0.340025,25.079,1.414,0.484,0.712352,...,-0.490328,4.279203,-1.256709,-0.435227,-0.435227,-1.298871e-15,0.00,0,9.530756,-47.474752
HKSE:00867,0.078396,0.112613,0.992924,772.148,-0.116277,0.088496,4.250,0.373,0.424,0.623059,...,0.740033,5.600820,-0.116277,0.577058,0.577058,-2.280019e-15,0.50,1,8.250171,-1.604288
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HKSE:02158,0.000000,0.000000,-0.480927,537.979,0.000000,0.000000,-4.570,0.000,0.000,-0.338109,...,0.977669,-11.231397,0.000000,0.680269,0.680269,0.000000e+00,0.00,0,-13.143084,-62.499998
HKSE:06160,-0.092390,-0.098814,-8.851863,1878.504,0.065187,-0.088519,9.371,0.000,0.000,-0.775035,...,0.948572,-12.416455,0.030473,0.707317,0.707317,-9.992007e-16,0.75,1,-14.986863,-35.000008
HKSE:02171,0.000000,0.000000,-0.554001,81.788,0.000000,0.000000,-1.437,0.000,0.000,-2.456207,...,0.985501,-13.209316,0.000000,0.680269,0.680269,0.000000e+00,0.00,0,-15.121002,-31.685099
HKSE:09939,0.000000,0.000000,-0.910249,65.318,0.000000,0.000000,1.114,0.000,0.000,-1.369673,...,1.030241,-14.544245,0.000000,0.680269,0.680269,0.000000e+00,0.00,0,-16.455931,-19.827174


In [12]:
max_dd_list

[-71.0447744410721,
 -63.15789490195192,
 -66.38395965716538,
 -61.42741471192256,
 -89.7797613647995]